<a href="https://colab.research.google.com/github/stefanusaw/GenAI_With_AzureML/blob/main/Project_ImplementCloudwTransformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 23.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.4 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

ds = load_dataset("Helsinki-NLP/opus-100", "en-id")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/57.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/132k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
print(ds)

DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 1000000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
})


In [ ]:
ds['train'][0]

{'translation': {'en': 'Hmm. _', 'id': 'Hmm.'}}

In [ ]:

train_subset = ds['train'].shuffle(seed=42).select(range(100, 400))
validation_subset = ds['validation'].shuffle(seed=42).select(range(10, 110))


print(f"Train subset size: {len(train_subset)}")
print(f"Validation subset size: {len(validation_subset)}")

Train subset size: 300
Validation subset size: 100


In [ ]:
import re

def clean_text(text):
    # Remove unwanted characters (you can add more specific patterns)
    text = re.sub(r'[^a-zA-Z0-9\s.,?!]', '', text)  # Allow only alphanumeric and punctuation
    return text.strip()

def has_repeated_phrase(text):
    # Check for repetitive patterns in the text
    return len(set(text.split())) == 1  # If all words in the text are the same, consider it repetitive

def is_quality_data(example):
    # 1. Clean the texts
    en_clean = clean_text(example['translation']['en'])
    id_clean = clean_text(example['translation']['id'])

    # 2. Remove examples where any text is too short or "dirty"
    if len(en_clean) < 5 or len(id_clean) < 5:
        return False  # Discard too short or "meaningless" texts

    # 3. Filter out repetitive phrases (like "_l-ley- _l-ley-" or exact repetition)
    if has_repeated_phrase(en_clean) or has_repeated_phrase(id_clean):
        return False

    # 4. Check if source and target are too similar (e.g., translations that don't change meaning)
    if en_clean == id_clean:  # If source and target are too similar, discard
        return False

    return True

# Apply the filtering function to your dataset
train_subset = train_subset.filter(is_quality_data)
validation_subset = validation_subset.filter(is_quality_data)

# Optionally, confirm the number of remaining examples
print(f"Filtered train subset size: {len(train_subset)}")
print(f"Filtered validation subset size: {len(validation_subset)}")

# Sample filtered data:
print(train_subset['translation'][3])
print(validation_subset['translation'][3])


Filter:   0%|          | 0/257 [00:00<?, ? examples/s]

Filter:   0%|          | 0/97 [00:00<?, ? examples/s]

Filtered train subset size: 257
Filtered validation subset size: 97
{'en': "But you can't stay here.", 'id': 'Tapi kau tak bisa tinggal disini.'}
{'en': 'Are we sparring or what?', 'id': 'Ini membosankan atau apa?'}


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-id")
tokenizer = AutoTokenizer.from_pretrained("t5-small")
# model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-id")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")


def preprocess_function(examples):
    # Tokenize the source (English) and target (Indonesian) texts
    inputs = tokenizer([example['en'] for example in examples['translation']], truncation=True, padding="max_length", max_length=128)
    targets = tokenizer([example['id'] for example in examples['translation']], truncation=True, padding="max_length", max_length=128)

    # Set the tokenized target as labels
    inputs["labels"] = targets["input_ids"]

    return inputs

# Apply preprocessing to the train and validation subsets
train_subset = train_subset.map(preprocess_function, batched=True)
validation_subset = validation_subset.map(preprocess_function, batched=True)

# Confirm the processed columns
print(f"Processed train subset columns: {train_subset.column_names}")
print(f"Processed validation subset columns: {validation_subset.column_names}")

Map:   0%|          | 0/257 [00:00<?, ? examples/s]

Map:   0%|          | 0/97 [00:00<?, ? examples/s]

Processed train subset columns: ['translation', 'input_ids', 'attention_mask', 'labels']
Processed validation subset columns: ['translation', 'input_ids', 'attention_mask', 'labels']


In [ ]:
import os

# Disable WandB logging globally
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
# import evaluate

# bleu = evaluate.load("bleu")

# def compute_bleu(eval_pred):
#     # Extract the predictions and references from eval_pred
#     predictions, labels = eval_pred

#     # Decode the predictions from token IDs to text
#     predictions = [tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions]

#     # Decode the references from token IDs to text (BLEU expects a list of references for each prediction)
#     references = [[tokenizer.decode(ref, skip_special_tokens=True)] for ref in labels]

#     # Compute the BLEU score
#     return bleu.compute(predictions=predictions, references=references)



training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit = 5,
    logging_dir='./logs',
    logging_strategy="epoch",
    num_train_epochs=8,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    learning_rate=2e-5,
    weight_decay=0.01,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=validation_subset,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
)

trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,0.854300,0.806360
2,0.809800,0.793511
3,0.793400,0.785921
4,0.778900,0.779285
5,0.795100,0.775428
6,0.774000,0.771176
7,0.764600,0.769151
8,0.772600,0.768431


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=264, training_loss=0.7928393609596022, metrics={'train_runtime': 39.791, 'train_samples_per_second': 51.67, 'train_steps_per_second': 6.635, 'total_flos': 69565685956608.0, 'train_loss': 0.7928393609596022, 'epoch': 8.0})

In [ ]:
# import evaluate

# bleu = evaluate.load("bleu")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Load the trained model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("./results/checkpoint-304/")  # Replace with your model path
tokenizer = AutoTokenizer.from_pretrained("t5-small")  # You can use the same tokenizer as before

# Function for translation/inference
def translate(text, max_length=128):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=max_length)

    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            max_length=max_length,
            num_beams=4,  # Try increasing/decreasing the number of beams
            temperature=1.0,  # Lower the temperature for less diversity, or increase for more diversity
            top_k=50,  # Top-k sampling, experiment with different values
            early_stopping=True
        )

    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translation


# Test the translation (or generation) with a sample sentence
input_text = "Hello, how are you?"
translated_text = translate(input_text)
print(f"Input: {input_text}")
print(f"Translated: {translated_text}")


Input: Hello, how are you?
Translated: 
